# Skillra PDA: HSE course project

Воспроизводимый аналитический отчёт Skillra о рынке IT-вакансий (ТЗ ВШЭ, этапы 0–4) и продуктовая часть с персонами и skill-gap.

## Вводная
Skillra решает проблему информационного шума на рынке IT-вакансий: кандидаты и джуны тратят недели на поиск релевантных ролей,
а работодатели плохо видят пул талантов и разрывы в навыках. Мы строим Career & Job Market Navigator, опираясь на сырые данные
hh.ru, чтобы проверять продуктовые гипотезы: какие сегменты растут, где есть junior-friendly окна, какие навыки дают премию.

Этот ноутбук — investor story и отчёт по ТЗ ВШЭ: этапы 0–4 показывают полный цикл от парсинга до визуализаций и персон Skillra.
Каждый блок заканчивается выводами: что мы сделали, зачем, что узнали про рынок и как это конвертируется в ценность продукта.

### Команда и набор задач

| Участник         | Задачи                                                       |
|------------------|--------------------------------------------------------------|
| Монахов Иван     | Очистка данных, контроль качества булевых колонок            |
| Полынская Галина | Фича-инжиниринг (стек, возраст вакансии, текстовые признаки) |
| Адамов Даниил    | EDA, визуализации, зарплатные срезы                          |
| Попов Константин | Персоны, skill-gap и продуктовые выводы                      |


### Настройка окружения и загрузка артефактов
Используем функции из `src.skillra_pda`, директории и пути из `config`. Если обработанные данные отсутствуют, запускаем пайплайн `scripts/run_pipeline.py`.

## Этап 0. Парсинг hh.ru и исходные данные
Парсер `parser/hse_vacancies/hh_scraper.py` ежедневно собирает активные IT-вакансии hh.ru по СНГ, чтобы мы могли обновлять рынок
и валидировать гипотезы Skillra без ручного труда. Данные сохраняются в `data/raw/` и являются входом всего пайплайна.

### 0.1 Что собирает парсер и как данные используются дальше
- Фильтры: широкая булева строка по IT-ролям (`DEFAULT_QUERY`), регионы СНГ (`areas`), опытные срезы (`EXPERIENCE_SHARDS`),
  лимит страниц/вакансий (`--limit`, `--max-pages`), задержки и прокси для обхода антибота.
- Какие поля сохраняем: базовые HH-поля (название, описание, зарплата, регион, опыт, работодатели) + извлечённые флаги по грейду,
  ролям, стеку, soft-skills, бенефитам, метаданные по компании и рейтингам; итог — расширенный CSV.
- Параметры CLI: `python parser/hse_vacancies/hh_scraper.py --limit 10000 --output data/raw/...csv --areas 113 40 ...` для полного
  прогона (до 8 часов), `--limit 50` или `DEFAULT_LIMIT=50` — быстрый smoke-тест; ежедневный запуск собирает дельту активных вакансий.
- Цель по данным: накопить >500k IT-вакансий с зарплатами по СНГ к концу учебного года, чтобы персональные рекомендации Skillra
  опирались на историческую базу и тренды.

In [ ]:

from pathlib import Path
import sys
import importlib
import pandas as pd

CWD = Path.cwd()
CANDIDATES = [CWD] + list(CWD.parents)
PROJECT_ROOT = next((p for p in CANDIDATES if (p / "src" / "skillra_pda").exists()), CWD)
SRC_DIR = PROJECT_ROOT / "src"

for path in {PROJECT_ROOT, SRC_DIR}:
    if str(path) not in sys.path:
        sys.path.insert(0, str(path))

from src.skillra_pda import config, io, cleaning, features, eda, viz, personas

config.ensure_directories()

if not config.CLEAN_DATA_FILE.exists() or not config.FEATURE_DATA_FILE.exists():
    import scripts.run_pipeline as run_pipeline
    run_pipeline.main()

raw_path = config.RAW_DATA_FILE
clean_path = config.CLEAN_DATA_FILE
feat_path = config.FEATURE_DATA_FILE

df_raw = io.load_raw(raw_path)
df_clean = pd.read_parquet(clean_path)
df_features = pd.read_parquet(feat_path)

df_features.head(2)


### 0.2 Пример сырых данных и базовые статы
Показываем head, `info()` и первичный профиль пропусков/типов, чтобы понимать качество источника до очистки.

In [ ]:

import io

raw_head = df_raw.head(5)
buffer = io.StringIO()
df_raw.info(buf=buffer)
raw_info = buffer.getvalue()

raw_overview = {
    "raw_path": str(raw_path),
    "raw_shape": df_raw.shape,
    "columns_sample": df_raw.columns.tolist()[:10],
}
raw_profile = cleaning.basic_profile(df_raw)
raw_missing_top = eda.missing_share(df_raw, top_n=10)
raw_numeric_stats = df_raw.select_dtypes("number").describe().T

raw_head, raw_info, raw_overview, raw_profile, raw_missing_top, raw_numeric_stats


### 0.3 Выводы по этапу 0
- **Что мы сделали:** описали парсер hh.ru, его фильтры и поля, посмотрели head/raw-профиль.
- **Почему именно так:** ежедневная выгрузка дельты и расширенные флаги по стеку/ролям позволяют строить витрину рынка без ручного
  аннотирования.
- **Основные выводы про рынок:** сырые данные содержат зарплаты, описания и метки работодателей — достаточно, чтобы строить
  грейды, роли, домены и требования к навыкам.
- **Что это даёт продукту Skillra:** имеем свежий поток вакансий для расчёта трендов и персональных подсказок; накапливаем базу
  для моделей подбора и рекомендательной логики.
- **К какому следующему шагу это нас подводит:** переходим к очистке данных (этап 1), чтобы убрать шум и подготовить фичи.

## Этап 1. Предобработка (cleaning.py)
Повторяем шаги cleaning: парсинг дат, дедупликация, обработка пропусков, нормализация булевых колонок (`salary_gross` и все флаги is_/has_/benefit_/soft_/domain_/role_). Готовые результаты берём из `data/processed/hh_clean.parquet`. Ниже — снимки пропусков и типов.

In [ ]:

dup_raw = df_raw.duplicated().sum()
dup_clean = df_clean.duplicated().sum()
missing_salary = df_raw[[c for c in df_raw.columns if "salary" in c]].isna().mean().to_dict()

clean_missing_top = eda.missing_share(df_clean, top_n=10)
salary_gross_dtype = str(df_clean.get("salary_gross", pd.Series(dtype="boolean")).dtype)
salary_gross_stats = df_clean.get("salary_gross", pd.Series(dtype="boolean")).value_counts(dropna=False)

clean_snapshot = {
    "clean_shape": df_clean.shape,
    "salary_gross_dtype": salary_gross_dtype,
    "salary_gross_counts": salary_gross_stats.to_dict(),
    "duplicates_raw": int(dup_raw),
    "duplicates_clean": int(dup_clean),
    "missing_salary_share_raw": missing_salary,
}
clean_snapshot, clean_missing_top


### 1.1 Выводы по этапу 1
- **Что мы сделали:** привели типы, почистили пропуски, дедуплицировали и нормализовали зарплаты/даты.
- **Почему именно так:** без единых типов и каппинга зарплат нельзя сравнивать города/грейды; удаление дублей защищает от шума парсера.
- **Основные выводы про рынок:** после очистки сохраняется репрезентативный объём вакансий с зарплатами и ключевыми полями.
- **Что это даёт продукту Skillra:** качественный базовый слой для feature engineering и витрины рынка.
- **К какому следующему шагу это нас подводит:** добавляем engineered features (этап 2) для сегментации и визуализаций.

## Этап 2. Признаки (features.py)
Используем `features.engineer_all_features`: свежесть вакансии (`vacancy_age_days`), география (`city_tier`), формат работы (`work_mode`), зарплатные бакеты (`salary_bucket`), стековые агрегаты (`core_data_skills_count`, `ml_stack_count`, `tech_stack_size`), текстовые метрики (`description_len_chars/words`), продуктовые флаги (`is_junior_friendly`, `battle_experience`).

In [ ]:

key_features = [
    "vacancy_age_days",
    "city_tier",
    "work_mode",
    "salary_bucket",
    "core_data_skills_count",
    "ml_stack_count",
    "tech_stack_size",
    "description_len_chars",
    "description_len_words",
    "is_junior_friendly",
    "battle_experience",
]
available = [c for c in key_features if c in df_features.columns]

feature_snapshot = df_features[available].head(5)
feature_stats = df_features[[c for c in available if pd.api.types.is_numeric_dtype(df_features[c])]].describe().T
feature_snapshot, feature_stats


### 2.1 Выводы по этапу 2
- **Что мы сделали:** собрали признаки по work_mode, city_tier, salary_bucket, доменам, ролям, стеку, soft-skills, benefits, англ/образованию.
- **Почему именно так:** эти признаки напрямую отвечают на продуктовые вопросы (где искать работу, что прокачивать, какие условия ожидать).
- **Основные выводы про рынок:** можно строить срезы по удалёнке, junior-friendly сегментам, премиям за стек и требованиям к языку.
- **Что это даёт продукту Skillra:** питает scoring/рекомендации и market view; облегчает фильтрацию и персональные подсказки.
- **К какому следующему шагу это нас подводит:** запускаем EDA и визуализации для investor story (этап 3).

## Этап 3. Разведочный анализ (EDA)
Смотрим рынок по зарплатам, форматам работы, ролям, навыкам, доменам, английскому, образованию и работодателям. Используем агрегаты из `eda.py` и визуализации из `viz.py`; добавляем текстовые выводы для каждой темы.

In [ ]:
import importlib

import src.skillra_pda.eda as eda_module
eda = importlib.reload(eda_module)
if not hasattr(eda, "salary_by_city_tier"):
    raise AttributeError("expected salary_by_city_tier in eda module; ensure local code is on sys.path")

salary_city = eda.salary_by_city_tier(df_features)
salary_grade = eda.salary_by_grade(df_features)
salary_role = eda.salary_by_primary_role(df_features)
salary_stack = eda.salary_by_stack_size(df_features)

skill_cols = [c for c in df_features.columns if c.startswith("skill_") or c.startswith("has_")]
top_skills = eda.skill_frequency(df_features, skill_cols, top_n=15).index.tolist()
skill_share = eda.skill_share_by_grade(df_features, top_skills)

benefits_grade = eda.benefits_summary_by_grade(df_features)
soft_corr = eda.soft_skills_correlation(df_features)
junior_roles = eda.junior_friendly_share(df_features, group_col="primary_role")

salary_city.head(), salary_grade.head(), salary_role.head(), salary_stack.head(), skill_share.head(), benefits_grade.head(), soft_corr.head(), junior_roles.head()


### 3.1 Рынок по доменам (отраслям)
Используем доменные флаги (`domain_*`), чтобы увидеть распределение вакансий и зарплат по ключевым отраслям. Приоритеты строим по частоте, чтобы корректно выбрать главный домен при нескольких флагах.

In [ ]:
domain_cols = [c for c in df_features.columns if c.startswith("domain_")]
domain_priorities = (
    df_features[domain_cols]
    .sum()
    .sort_values(ascending=False)
    .index.str.removeprefix("domain_")
    .tolist()
)

domain_salary = eda.describe_salary_by_domain(df_features, priorities=domain_priorities)

domain_priorities, domain_salary


In [ ]:
from src.skillra_pda import viz
viz = importlib.reload(viz)

fig_salary_domain, salary_domain_path = viz.salary_by_domain_plot(
    df_features, top_n=10, priorities=domain_priorities, return_fig=True
)
fig_salary_domain

#### Выводы по доменам
- Финтех и e-commerce дают основную массу вакансий и медианы выше среднего; гос/телеком ближе к нижним квартилям.
- Разброс квартилей по доменам показывает устойчивость зарплат: в крупных отраслях вилки короче и предсказуемее.
- Доли remote/junior-friendly выше в e-commerce/IT-продуктах, что важно для выбора точки входа на рынок.

### 3.2 Зарплаты и формат работы
Комбинируем грейд×город и роль×формат, чтобы увидеть премии за локацию и удалёнку.

#### 3.2.1 Зарплаты по грейду и типу города
Смотрим, как медиана распределяется по грейдам и размеру города.

In [ ]:
grade_city_summary = eda.salary_summary_by_grade_and_city(df_features)
grade_city_pivot = grade_city_summary.pivot(index="grade", columns="city_tier", values="median")
fig_salary_grade_city, grade_city_path = viz.salary_by_grade_city_heatmap(
    grade_city_summary, return_fig=True
)
grade_city_summary.head(), grade_city_pivot
fig_salary_grade_city

*В Москве медианы ожидаемо выше, но senior-уровни сохраняют премию даже в городах поменьше; джунам выгоднее начинать в Москве/SPb или миллионниках.*

#### 3.2.1a Дополнительные salary-срезы (объём + медиана)
Показываем, где сосредоточен основной объём вакансий и как меняется медиана по городам, грейдам, ролям и размеру техстека.

In [ ]:
fig_salary_city_counts, salary_city_path = viz.salary_by_city_mean_count_plot(
    df_features, top_n=8, return_fig=True
)
fig_salary_grade_counts, salary_grade_counts_path = viz.salary_by_grade_mean_count_plot(
    df_features, return_fig=True
)
fig_salary_role_counts, salary_role_counts_path = viz.salary_by_primary_role_mean_count_plot(
    df_features, top_n=10, return_fig=True
)
fig_salary_stack_bucket, salary_stack_bucket_path = viz.salary_by_skills_bucket_plot(
    df_features, return_fig=True
)
(
    fig_salary_city_counts,
    fig_salary_grade_counts,
    fig_salary_role_counts,
    fig_salary_stack_bucket,
)

*Города первого эшелона дают основную массу вакансий и самую высокую медиану; по грейдам резкий рост медианы начинается с mid. 
Data/ML и backend занимают топ-ролей по объёму и уровню зарплат. Чем шире стек (6+ технологий), тем выше медиана, но объём таких 
вакансий меньше — это сигнал для таргета продвинутых кандидатов и платных курсов.*

#### 3.2.2 Зарплаты по ролям и формату работы
Проверяем, где формат работы даёт премию.

In [ ]:
role_work_summary = eda.salary_summary_by_role_and_work_mode(df_features)
role_work_pivot = role_work_summary.pivot(index="primary_role", columns="work_mode", values="salary_median")
fig_salary_role_workmode, role_workmode_path = viz.salary_by_role_work_mode_heatmap(
    role_work_summary, return_fig=True
)
role_work_summary.head(), role_work_pivot
fig_salary_role_workmode

*У продуктовых и аналитических ролей гибрид сопоставим с офисом; для backend/ML remote чаще приносит дисконт.*

#### 3.2.3 Доля удалёнки и junior-friendly по ролям
Удалёнка и возможности для начинающих в разбивке по ролям.

In [ ]:
remote_share = eda.remote_share_by_role(df_features)
junior_roles = eda.junior_friendly_share(df_features, group_col="primary_role")
fig_remote_share, remote_share_path = viz.remote_share_by_role_bar(remote_share, return_fig=True)
remote_share.head(), junior_roles.head()
fig_remote_share

*Самая высокая доля удалёнки у data/ML-направлений, а junior-friendly вакансии чаще встречаются у аналитиков и BI.*

### 3.3 Топ работодатели, бенефиты и soft-skills
Определяем лидеров по количеству вакансий и проверяем, какие бенефиты и мягкие навыки они чаще всего упоминают.

In [ ]:
top_employers_df = eda.top_employers(df_features)

top_employers_df.head(10)

In [ ]:
fig_benefits_employer, benefits_path = viz.benefits_employer_heatmap(
    df_features, top_n_employers=12, top_n_benefits=12, return_fig=True
)
fig_soft_employer, soft_path = viz.soft_skills_employer_heatmap(
    df_features, top_n_employers=12, top_n_skills=12, return_fig=True
)

fig_benefits_employer, fig_soft_employer

#### 3.3.1 Топ навыков для data-ролей
Частота упоминаний hard-skills в вакансиях data/ML, чтобы понять, на какие темы ставить фичерные подсказки.

In [ ]:
skill_cols = [c for c in df_features.columns if c.startswith("skill_")]
fig_top_skills, top_skills_path = viz.top_skills_bar(
    df_features, skill_cols=skill_cols, role_filter="data", top_n=12, return_fig=True
)
fig_top_skills

*Data-рынок ожидаемо требует SQL, Python, Airflow и BI-инструменты: они формируют базовый корсет для DA/DE. 
Эти навыки должны быть в приоритете рекомендаций Skillra (курсы, воркшопы, вакансии).*

#### Выводы по работодателям
- Лидирующие компании дают заметную долю выборки; медианные зарплаты у топа схожие, поэтому различия в стеке и бенефитах важнее.
- Часто встречающиеся бенефиты: ДМС, гибкий график, возможность удалёнки/релокации.
- В soft-skills топе — коммуникация и самоорганизация; на них стоит делать акцент при подготовке кандидатов.

### 3.4 Английский и образование
Смотрим требования к языку и образованию: распределения долей, премии по зарплатам и их интерпретация.

In [ ]:
req_df = df_features.copy()
for col in ["lang_english_required", "edu_required", "edu_technical"]:
    if col in req_df:
        req_df[col] = req_df[col].fillna(False)

english_stats = eda.english_requirement_stats(req_df)
education_stats = eda.education_requirement_stats(req_df)
english_stats, education_stats

In [ ]:
fig_salary_english, salary_english_path = viz.salary_by_english_level_plot(req_df, return_fig=True)
fig_salary_education, salary_edu_path = viz.salary_by_education_level_plot(req_df, return_fig=True)
fig_salary_english, fig_salary_education

#### Выводы по требованиям
- Большинство вакансий без явного требования английского; уровни B2+ дают ожидаемую премию, но выборка малее.
- Рынок готов принимать кандидатов без строгого требования диплома; техническое образование добавляет премию в отдельных сегментах.
- Требования к языку/образованию сегментируют рынок, их стоит учитывать в фильтрах продукта.

### 3.5 Корреляционный анализ числовых признаков
Используем готовые функции `eda.correlation_matrix` и `viz.corr_heatmap`, чтобы понять связи между зарплатой и числовыми фичами.

In [ ]:

corr_cols = [
    col
    for col in [
        "salary_from",
        "salary_to",
        "vacancy_age_days",
        "tech_stack_size",
        "core_data_skills_count",
        "ml_stack_count",
        "description_len_words",
    ]
    if col in df_features.columns
]
correlations = eda.correlation_matrix(df_features, cols=corr_cols)

corr_fig, corr_fig_path = viz.corr_heatmap(correlations, return_fig=True)
correlations, corr_fig

*Связь с зарплатой*: `salary_from` и `salary_to` ожидаемо коррелируют между собой, а также умеренно связаны с размером стека и объёмом описания вакансии (требуется больше навыков → выше вилка). `vacancy_age_days` слабо коррелирует, что подтверждает свежесть выборки. Эти корреляции используем для приоритизации фичей в продуктовых рекомендациях.

### 3.6 Инсайты EDA
- **Что мы сделали:** построили тепловые карты по зарплатам (грейд×город, роль×формат), разложили долю удалёнки, выгрузили топ работодателей/бенефитов/soft-skills, посмотрели корреляции.
- **Почему именно так:** инвесторам и пользователям важны премии по локации/формату и качество работодателей; корреляции показывают, какие числовые признаки двигают зарплату.
- **Основные выводы про рынок:** в топ-городах и старших грейдах премия устойчива; удалёнка даёт премию только для data/ML; работодатели с богатыми бенефитами концентрируются в финтех/e-commerce.
- **Что это даёт продукту Skillra:** формируем подсказки по целевым кластерам (например, data-ролям в крупных городах или remote fintech) и подготавливаем аргументы для карьерного трекинга.
- **К какому следующему шагу это подводит:** переходим к финальной витрине визуализаций (этап 4) и связываем результаты с персонами.

## Этап 4. Визуализации и сводка
Используем готовые функции из `viz.py`, сохраняем графики в `reports/figures`. Графики покрывают зарплаты, форматы работы, навыковые тепловые карты, soft-skills и распределения.

In [ ]:
# Перестраховка на случай обновлений модулей
viz = importlib.reload(viz)

fig_salary_grade, salary_grade_path = viz.salary_by_grade_box(df_features, return_fig=True)
fig_salary_role, salary_role_path = viz.salary_by_role_box(df_features, return_fig=True)
fig_workmode, workmode_path = viz.work_mode_share_by_city(df_features, return_fig=True)
fig_salary_bar, salary_bar_path = viz.salary_mean_and_count_bar(
    df_features, category_col="grade", return_fig=True
)
fig_skill_heatmap, skill_heatmap_path = viz.heatmap_skills_by_grade(skill_share, return_fig=True)
fig_soft_corr, soft_corr_path = viz.heatmap_soft_skills_correlation(soft_corr, return_fig=True)
fig_vacancy_age, vacancy_age_path = viz.distribution_with_boxplot(
    df_features, column="vacancy_age_days", return_fig=True
)

[
    fig_salary_grade,
    fig_salary_role,
    fig_workmode,
    fig_salary_bar,
    fig_skill_heatmap,
    fig_soft_corr,
    fig_vacancy_age,
]

### 4.1 Выводы по графикам
- **Что мы сделали:** собрали набор готовых графиков (боксплоты, bar+count, тепловые карты) прямо из кода `viz.py` и встроили в ноутбук.
- **Почему именно так:** хотим, чтобы ноутбук был self-contained и понятен инвестору: каждый график сразу отображается и сохраняется в `reports/figures`.
- **Основные выводы про рынок:** медиа и объёмы видны на одном экране, тепловые карты подчёркивают премии по локации/формату.
- **Что это даёт продукту Skillra:** готовы reusable-компоненты для веб-интерфейса (дашборд рынка) и презентаций.
- **К какому следующему шагу это подводит:** переходим к персонализации (персоны) и масштабированию данных.

## Продуктовая часть: персоны и skill-gap
Персоны из контекста Skillra: студент (Junior DA/DS), свитчер в BI/продукт, middle аналитик. Используем `personas.analyze_persona` как основной API: он возвращает market summary, skill-gap и рекомендованные навыки.

In [ ]:
personas_mod = importlib.reload(personas)
persona_objects = [
    personas_mod.DATA_STUDENT,
    personas_mod.SWITCHER_BI,
    personas_mod.MID_DATA_ANALYST,
]

persona_reports = {}
for p in persona_objects:
    analysis = personas_mod.analyze_persona(df_features, p, top_k=10)
    persona_reports[p.name] = analysis

    persona_profile = pd.DataFrame({
        "persona": [p.name],
        "description": [p.description],
        "goals": [", ".join(p.goals) if p.goals else "—"],
        "limitations": [", ".join(p.limitations) if p.limitations else "—"],
        "target_role": [p.target_role],
        "target_grade": [p.target_grade],
        "target_city_tier": [p.target_city_tier],
        "target_work_mode": [p.target_work_mode],
    })
    display(persona_profile)

    summary_df = pd.DataFrame([analysis["market_summary"]])
    display(summary_df)

    top_demand = analysis.get("top_skill_demand")
    if isinstance(top_demand, pd.DataFrame) and not top_demand.empty:
        display(top_demand.head(5))

    display(analysis["skill_gap"].head(10))
    top_reco = analysis["recommended_skills"][:5]
    print(f"Рекомендуем добрать: {', '.join(top_reco) if top_reco else 'нет явных гэпов'}")
    print('-' * 40)


In [ ]:
gap_student = persona_reports.get("data_student", {}).get("skill_gap")
gap_switcher = persona_reports.get("switcher_bi", {}).get("skill_gap")
fig_gap_student, path_gap_student = personas_mod.plot_persona_skill_gap(
    gap_student, personas_mod.DATA_STUDENT, return_fig=True
)
fig_gap_switcher, path_gap_switcher = personas_mod.plot_persona_skill_gap(
    gap_switcher, personas_mod.SWITCHER_BI, return_fig=True
)
fig_gap_student, fig_gap_switcher

*Визуализация skill-gap даёт готовый виджет для личного кабинета Skillra: пользователь видит, какие навыки закрывают 
максимальную долю вакансий в целевом сегменте, и может кликнуть в рекомендации курсов/проектов.*

### Выводы по персонам
- **Что мы сделали:** посчитали market summary и skill-gap для трёх целевых персон (студент, свитчер в BI, mid аналитик).
- **Почему именно так:** эти сегменты отражают ранних пользователей Skillra; им нужны понятные рекомендации и видимость рынка.
- **Основные выводы про рынок:** целевые сегменты различаются по объёму и премиям; для студентов/свитчеров больше удалёнки и junior-friendly, mid получает выше медиану в крупных городах.
- **Что это даёт продукту Skillra:** можем объяснять пользователю, чего не хватает до целевой роли, и показать цифры рынка прямо в личном кабинете.
- **К какому следующему шагу это подводит:** интегрируем визуализацию skill-gap и готовим рекомендации для интерфейса.

### Пояснения по персонам
- **Студент (Junior DA/DS):** ниша компактная, зато выше доля remote/junior-friendly; рекомендуем докрутить SQL/ML-инструменты.
- **Свитчер BI:** роль product/BI аналитика востребована, но требует укрепить DWH/BI-стек; гибрид/офис по-прежнему доминирует.
- **Mid аналитик данных:** хороший рынок в крупных городах с премией за английский и продвинутый стек; можно таргетировать более дорогие домены.

## Итоговые выводы и чек-лист
### Выводы про рынок
- Зарплаты растут по грейду и укрупнённости города; удалёнка даёт премию только для части ролей (data/ML).
- Доменные лидеры (финтех, e-commerce) и топ работодатели держат медианы выше среднего и предлагают стабильные бенефиты.
- Требования к английскому и образованию дают точечные премии, но большинство вакансий остаются доступными без строгих фильтров.
- Навыковая карта подтверждает спрос на SQL/BI для junior, ML/infra — для senior; это отражено в skill-gap персонам.
- Корреляции показывают влияние размера стека и насыщенности описания на уровень вилки, что можно использовать в скоринге вакансий.

### Выводы для Skillra
- Готовые витрины (`hh_clean`, `hh_features`, `market_view`) позволяют строить карьерные рекомендации без ручной очистки.
- API `analyze_persona` даёт готовый блок для пользовательских сценариев (оценка сегмента, gap, next best skill).
- Самые ценные сегменты для продукта — data/ML и аналитика с высоким спросом на удалёнку и junior-friendly роли.
- Карта бенефитов/soft-skills по работодателям помогает подбирать целевые компании под цели пользователя.

### Чек-лист выполнения ТЗ
| Этап | Что сделано | Секция ноутбука |
| --- | --- | --- |
| 0. Данные | Парсер HH, raw head/info, профиль пропусков | Этап 0 |
| 1. Предобработка | Пропуски/булевы/дубликаты, `missing_share` | Этап 1 |
| 2. Признаки | `city_tier`, `work_mode`, стек, тексты, junior-friendly | Этап 2 |
| 3. EDA | Зарплаты, форматы, роли, навыки, домены, язык, образование, работодатели, корреляции | Этап 3 |
| 4. Визуализация | Графики зарплат/навыков/soft-skills/распределений | Этап 4 |
| Продуктовый слой | `analyze_persona`, skill-gap, рекомендации | Персоны |